In [ ]:
!pip install arxiv

In [ ]:
%pip install langchain_community

# This is a inbuilt tool from langchain

In [3]:
# Create an Wrapper on top of Wikipedia tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# For Wikepedia, we don't need any seperate api key, already langchain is taking care of it

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200) # this is interacting with the Wikipedia

Wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
Wiki.name

'wikipedia'

# Load the environment variable into this virtual environment from the .env file

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Now we will take a website and read all its contents as data source. This is a custom tool

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the documents and stored in VectorDB
loader = WebBaseLoader("https://docs.smith.langchain.com/")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

# Create Vector Store
vectorDB = FAISS.from_documents(
    docs,
    embeddings
)

retriever = vectorDB.as_retriever()

retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022571506CF0>)

In [11]:
# Creating a tool to do retrieval of documents from a vectorDB

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "Langsmith-Search",
    "search for information about Langsmith. For any question about langsmith you must use this tool"
)

In [12]:
retriever_tool.name

'Langsmith-Search'

# Loading another Inbuilt tool "arxiv" from Langchain

In [14]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=200
)

arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [15]:
arxiv.name

'arxiv'

In [18]:
# Now we have the three tools with us combine them

tools = [Wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Labs_Building_Generative_AI_Applications\\RAG_Agent\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Langsmith-Search', description='search for information about Langsmith. For any question about langsmith you must use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022553C83920>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langc

# Create LLM

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
        model="gpt-3.5-turbo-0125",
        temperature=0
    )

In [30]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022583BB7C20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022583BB9BB0>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

# To create Prompt, we use HUB

In [31]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Now we will initializae agents, which act as a reasoning engine, which uses ReAct Prompting technique for the LLM to decide which tool to use and in what order to use based on the user i/p. So it will sequentially excute with that tool from left to right.

In [32]:
from langchain.agents import create_openai_tools_agent

create_openai_tools_agent = create_openai_tools_agent(
    llm,
    tools,
    prompt
)

In [33]:
from langchain.agents import AgentExecutor

AgentExecutor = AgentExecutor(
    agent = create_openai_tools_agent,
    tools=tools,
    verbose=True
)

AgentExecutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [34]:
# Try to invoke the model and see the output

AgentExecutor.invoke({
    "input": "What is the ASU motto?"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Arizona State University motto'}`


Page: List of U.S. state and territory mottos
Summary: Most of the United States' 50 states have a state motto, as do the District of Columbia and 3 of its territories. A motto is a phrase intended toThe motto of Arizona State University is "Be the measure."

> Finished chain.


{'input': 'What is the ASU motto?',
 'output': 'The motto of Arizona State University is "Be the measure."'}

In [42]:
AgentExecutor.invoke({
    "input": "Tell me about Langsmith from langsmith page?"
})



> Entering new AgentExecutor chain...

Invoking: `Langsmith-Search` with `{'query': 'Langsmith'}`







Get started with LangSmith | 🦜️🛠️ LangSmith







Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log yo

{'input': 'Tell me about Langsmith from langsmith page?',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works on its own and does not require the use of LangChain. \n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using Python or TypeScript.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by exporting necessary variables like LANGCHAIN_TRACING_V2, LANGCHAIN_API_KEY, and OPENAI_API_KEY.\n4. Log your first trace by using traceable functions to log traces to LangSmith.\n5. Run your first evaluation by defining datasets, evaluators, and running experiments to test your AI system.\n\nFor more detailed information and tutorials on using LangSmith, you can visit the LangSmith page on the LangChain website.'}

In [40]:
AgentExecutor.invoke({
    "input": "Expllain the Types of machine learning algorithms?"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Types of machine learning algorithms'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning algorithms can be broadly categorized into three main types:

1. Supervised Learning: In supervised learning, the algorithm is trained on a labeled dataset, where each input data point is paired with the correct output. The algorithm learns to map inputs to outputs based on the labeled examples it is provided. Common supervised learning algorithms include linear regression, support vector machines, decision trees, and neural networks.

2. Unsupervised Learning: Unsupervised learning involves training the algorithm on an unlabeled dataset, where the algorithm must find patterns and relationships in the data without explicit guidance. Clustering algorithm

{'input': 'Expllain the Types of machine learning algorithms?',
 'output': 'Machine learning algorithms can be broadly categorized into three main types:\n\n1. Supervised Learning: In supervised learning, the algorithm is trained on a labeled dataset, where each input data point is paired with the correct output. The algorithm learns to map inputs to outputs based on the labeled examples it is provided. Common supervised learning algorithms include linear regression, support vector machines, decision trees, and neural networks.\n\n2. Unsupervised Learning: Unsupervised learning involves training the algorithm on an unlabeled dataset, where the algorithm must find patterns and relationships in the data without explicit guidance. Clustering algorithms, such as K-means clustering and hierarchical clustering, and dimensionality reduction techniques like principal component analysis (PCA) are examples of unsupervised learning algorithms.\n\n3. Reinforcement Learning: Reinforcement learning 

In [43]:
AgentExecutor.invoke({
    "input": "What is the paper 2407.17522 about?"
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2407.17522'}`


Published: 2024-07-20
Title: Mapping the Technological Future: A Topic, Sentiment, and Emotion Analysis in Social Media Discourse
Authors: Alina Landowska, Maciej Skorski, Krzysztof Rajda
Summary: PeoThe paper with identifier 2407.17522 is titled "Mapping the Technological Future: A Topic, Sentiment, and Emotion Analysis in Social Media Discourse" authored by Alina Landowska, Maciej Skorski, and Krzysztof Rajda. The paper focuses on analyzing topics, sentiments, and emotions in social media discourse related to mapping the technological future.

> Finished chain.


{'input': 'What is the paper 2407.17522 about?',
 'output': 'The paper with identifier 2407.17522 is titled "Mapping the Technological Future: A Topic, Sentiment, and Emotion Analysis in Social Media Discourse" authored by Alina Landowska, Maciej Skorski, and Krzysztof Rajda. The paper focuses on analyzing topics, sentiments, and emotions in social media discourse related to mapping the technological future.'}